# Implementation of Policy Iteration

#### *pymdptoolbox* is a python library from which MDP examples can be imported </br>
##### Documentation of *pymdptoolbox* : https://pymdptoolbox.readthedocs.io/en/latest/api/mdptoolbox.html

In [64]:
!pip install pymdptoolbox   #Install pymdptoolbox within the code

## Import Required Libraries

In [65]:
import mdptoolbox.example
import numpy as np
import sys
import time
import warnings
warnings.filterwarnings('ignore')

## List of Functions to be implemented

-Define environment </br>
-Policy Evaluation </br>
-Bellman Update </br>
-Choose Greedy action and update policy </br>
-Policy Improvement </br>
-Policy Iteration </br>

## Class MarkovDP 

In [66]:
'''
Class MarkovDP contains the following attributes:
1)Number of states  : s
2)Number of actions : a
3)State Space
4)Action Space
5)Transition probability matrix of size (a,s,s)
6)Reward matrix (a,s,s)
'''
class MarkovDP:
    def __init__(self,s,a):
        self.num_state             = s
        self.num_action            = a
        self.states                = np.array(range(0,s))
        self.actions               = np.array(range(0,a))
        self.transitions           = np.zeros((a,s,s))
        self.rewards               = np.zeros((a,s,s))
        
# The function below initializes transition probability matrix and rewards marks 

    def initialize_mdp(self):      
        np.random.seed(0)        #for reproducibility 
        self.transitions, self.rewards = mdptoolbox.example.rand(self.num_state,self.num_action)
        self.rewards = np.random.rand(self.num_action,self.num_state,self.num_state)

## Policy Evaluation Function

In [76]:
def evaluate_policy(env, V, pi, gamma, theta):
    while True:
        delta = 0
        for s in env.states:
            v = V[s].copy()
            V=update_v_policy(env, V, pi, s, gamma)    #Bellman update 
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    return V

## Bellman Update function
$$\large v(s) \leftarrow \sum_a \pi(a | s) \sum_{s', r} p(s', r | s, a)[r + \gamma v(s')]$$

In [77]:
def update_v(env, V, pi, s, gamma):
    sum=0
    for a in env.actions:
        transitions = np.reshape(env.transitions[a][s][:],(-1,1))
        rewards = np.reshape(env.rewards[a][s][:],(-1,1))
        sum=sum+pi[s][a]*(np.sum(np.multiply(transitions,rewards)+ gamma * np.multiply(transitions,V)))
    V[s]=sum
    return V


## Function that chooses the greedy action for a particular state 's'

In [78]:
def choose_best_action(env, V, pi, s, gamma):
    q=np.empty((env.num_action,1),dtype=float)
    for a in env.actions:
        pi[s][a]=0
        transitions = np.reshape(env.transitions[a][s][:],(-1,1))
        rewards = np.reshape(env.rewards[a][s][:],(-1,1))
        q[a]=np.sum(np.multiply(transitions,rewards)+ gamma * np.multiply(transitions,V))
    action=np.argmax(q)        #Choose greedy action
    pi[s][action]=1            #Update Policy


## Policy Improvement Function

In [79]:
def improve_policy(env, V, pi, gamma):
    policy_stable = True        # If policy_stable == True : Policy need not be updated anymore
    for s in env.states:
        old = pi[s].copy()
        choose_best_action(env, V, pi, s, gamma)
        if not np.array_equal(pi[s], old): 
            policy_stable = False
    return pi, policy_stable

## Policy Iteration Function

#### Initialize Value function vector : [0,0,0...0]
#### Initialize policy : Uniform Probability distribution

In [80]:
def policy_iteration(env, gamma, theta):
    V = np.zeros((env.num_state,1))          #Initialize Value function vector : [0,0,0...0]
    pi = np.ones((env.num_state,env.num_action)) / env.num_action   #Policy Initialization
    policy_stable = False
    while not policy_stable:
        V = evaluate_policy(env, V, pi, gamma, theta)          #Policy Evaluation step
        pi, policy_stable = improve_policy(env, V, pi, gamma)  #Policy Iteration step
    return V, pi

## Example

In [81]:
'''
Define an MDP Environment : Insantiate Class
    Number of states : 10
    Number of actions : 3
'''
env= MarkovDP(10,3)      #Define an MDP Environment : Insantiate Class
env.initialize_mdp()    #Define P and R

In [82]:
gamma = 0.9       #Discount rate
theta = 0.0001    #A small positive number

start_time = time.time()
V,pi=policy_iteration(env, gamma, theta)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.12379813194274902 seconds ---


In [83]:
#Final values of the value function at each state
for s in env.states:
    print('V(s',s+1,') :',V[s][0])

V(s 1 ) : 7.573231634060111
V(s 2 ) : 7.430471492387628
V(s 3 ) : 7.6552394629170335
V(s 4 ) : 7.362491475340923
V(s 5 ) : 7.678968623420015
V(s 6 ) : 7.574400188551869
V(s 7 ) : 7.4880983523263
V(s 8 ) : 7.557449501942633
V(s 9 ) : 7.94906786700345
V(s 10 ) : 7.909065762013673


In [84]:
#Optimal Policy
for s in env.states:
    print('P(a|s',s+1,')=',pi[s])

P(a|s 1 )= [1. 0. 0.]
P(a|s 2 )= [1. 0. 0.]
P(a|s 3 )= [1. 0. 0.]
P(a|s 4 )= [0. 0. 1.]
P(a|s 5 )= [0. 0. 1.]
P(a|s 6 )= [1. 0. 0.]
P(a|s 7 )= [1. 0. 0.]
P(a|s 8 )= [0. 0. 1.]
P(a|s 9 )= [0. 0. 1.]
P(a|s 10 )= [0. 1. 0.]
